In [1]:
# Set up dependencies
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import pandas as pd
import os
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
# Setup Chrome Driver for use with Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\mthor\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


## NASA Mars News

In [3]:
# Scrape this page: https://mars.nasa.gov/news/
# Collect the latest News Title and Paragraph Text
# Assign the text to variables that you can reference later.

# Example:
# news_title = "NASA's Next Mars Mission to Investigate Interior of Red Planet"
# news_p = "Preparation of NASA's next spacecraft to Mars, InSight, has ramped up this summer, on course for launch next May from Vandenberg Air Force Base in central California -- the first interplanetary launch in history from America's West Coast."


In [4]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

# HTML object
html = browser.html

# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html, 'html.parser')

In [5]:
# Scrape the title, teaser and date from the first list_text class div element on the page
article = soup.find('div', class_='list_text')
news_title = article.find('div', class_='content_title').text
news_teaser = article.find('div', class_='article_teaser_body').text
article_date = article.find('div', class_='list_date').text
    
# print article data
print('-----------------')
print(article_date)
print(news_title)
print(news_teaser)
print('-----------------')



-----------------
March 17, 2021
Another First: Perseverance Captures the Sounds of Driving on Mars
NASA’s newest rover recorded audio of itself crunching over the surface of the Red Planet, adding a whole new dimension to Mars exploration.
-----------------


## JPL Mars Space Images - Featured Image

In [6]:
# Visit this page https://www.jpl.nasa.gov/images?search=&category=Mars
# Use Splinter to navigate the site and find the image URL to the full size image for the current Featured Mars Image
# Assign the complete url string to a variable called featured_image_url

# Example:
# featured_image_url = 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16225_hires.jpg'


In [7]:
# Visit the JPL Mars Space Images page
url = 'https://www.jpl.nasa.gov/images?search=&category=Mars'
browser.visit(url)

# Parse the HTML on this page with Beautiful Soup
html = browser.html
soup = BeautifulSoup(html,'html.parser')

# Find the link to the first displayed Image on the page and navigate to it
image_page = soup.find('div', class_='SearchResultCard').a['href']
full_image_page = f"https://www.jpl.nasa.gov{image_page}"
browser.visit(full_image_page)

# Parse the image page with Beautiful Soup and find the url of the full size image
html = browser.html
soup = BeautifulSoup(html,'html.parser')
featured_image_url = soup.find("img", class_="BaseImage").attrs["src"] 
print(featured_image_url)


https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24470.width-1024.jpg


## Mars Facts

In [8]:
# Visit the Mars facts page: https://space-facts.com/mars/
# Use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
# Use Pandas to convert the data to a HTML table string

In [9]:
url = 'https://space-facts.com/mars/'
tables = pd.read_html(url)
mars_df = tables[0]
mars_df.columns = ['Fact','Detail']
mars_df
mars_facts_table = mars_df.to_html(justify='left',index=False)
mars_df.to_html('mars_facts_tbl.html',justify='left',index=False)

In [10]:
# Probably want to come back and apply some CSS to this table using table_id= after I've designed the 
# web page I am going to render this on

## Mars Hemispheres

In [11]:
# Visit the USGS Astrogeology site https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars
# Click on each of the links to the hemispheres in order to find the image URL to the full res image
# Save both the image URL string for the full res hemisphere image and the Hemisphere title containing the hemisphere name
# Use a Python dictionary to store the data using the keys img_url and title.
# Append the dictionary with the image URL string and the hemisphere title to a list.
# This list will contain one dictionary for each hemisphere


In [12]:
# Visit the  USGS Astrogeology site and get titles of and links to Hemisphere image pages
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

# Parse the HTML on this page with Beautiful Soup
html = browser.html
soup = BeautifulSoup(html,'html.parser')

# Retrieve the parent divs for all items
items = soup.find_all('div', class_='item')

link_list = []
title_list = []
# loop over results to get Hemisphere image info
for item in items:
    # scrape the image title and link 
    link = item.find('a', class_='itemLink').attrs["href"] 
    title = item.find('h3').text
    link_list.append(link)
    title_list.append(title)


In [13]:
# Iterate through the list of hemisphere image links and navigate to each individual page to scrape the image url
image_list = []
for item in link_list:
   
    #Navigate to new page
    article_page = f"https://astrogeology.usgs.gov{item}"
    browser.visit(article_page)
    
    # HTML object
    html = browser.html
    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')
        
    # Retrieve elements in the downloads div and get the url to the full image
    downloads = soup.find("div", class_="downloads")
    image_url = downloads.find('a').attrs["href"]
    image_list.append(image_url)
    
  
 # Create a list of hemisphere dictionaries
Cerberus = {'title':title_list[0], 'img_url':image_list[0]}
Schiaparelli = {'title':title_list[1], 'img_url':image_list[1]}
Syrtis = {'title':title_list[2], 'img_url':image_list[2]}
Valles = {'title':title_list[3], 'img_url':image_list[3]}
hemisphere_image_urls = [Cerberus, Schiaparelli, Syrtis, Valles]
# print(hemisphere_image_urls)


[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [14]:
# Create a dictionary containing all of my scraped Mars website data
Mars_dict = {
       'news_title': news_title,
       'news teaser': news_teaser,
       'featured_image_url': featured_image_url,
       'mars_facts_table': str(mars_facts_table),
       'hemisphere_image_urls': hemisphere_image_urls
    }
print ("Dictionary of scraped Mars data contains : " +  str(Mars_dict)) 

Dictionary of scraped Mars data contains : {'news_title': 'Another First: Perseverance Captures the Sounds of Driving on Mars', 'news teaser': 'NASA’s newest rover recorded audio of itself crunching over the surface of the Red Planet, adding a whole new dimension to Mars exploration.', 'featured_image_url': 'https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24470.width-1024.jpg', 'mars_facts_table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: left;">\n      <th>Fact</th>\n      <th>Detail</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n   